# A prototype workflow for geo-referenced burst (STAC)ks

In [ ]:
%load_ext autoreload
%autoreload 2

import requests
import json
from pathlib import Path

from s1bursts import utils, metadata, download, opera

In [ ]:
asf_url = 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20211111T002553_20211111T002623_029532_038657_AB46.zip'

In [ ]:
auth = utils.get_netrc_auth()
manifest, annotations = metadata.edl_download_metadata(asf_url, auth)

In [ ]:
slc = metadata.SLCMetadata(asf_url, manifest, annotations)
swath = metadata.SwathMetadata(slc,'vv',swath_index=0)
burst = metadata.BurstMetadata(swath,burst_index=0)
item = burst.to_stac_item()
item.properties

In [ ]:
item.validate()

In [ ]:
catalog_url = 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS'
catalog = json.loads(requests.get(catalog_url).content)
len(catalog['items'])

In [ ]:
ids = [x['meta']['native-id'] for x in catalog['items']]
cmr_urls = [f'{catalog_url}&granule_ur={x}' for x in ids]
cmr_urls

In [ ]:
cmr_url = cmr_urls[0]

In [ ]:
burst_instance = opera.cmr_to_opera_burst(cmr_url,remote=True)

In [ ]:
burst_instance

In [ ]:
scratch_dir = Path('scratch')
if not scratch_dir.exists():
    scratch_dir.mkdir()

In [ ]:
burst_instance.slc_to_file(scratch_dir / 'not_georeferenced.tif',fmt='GTiff')

In [ ]:
!gdalinfo ./scratch/not_georeferenced.tif

In [ ]:
dem_path = Path('./compass_outputs/input_data/dem_galapagos.tif').resolve()
output_dir = scratch_dir / 'geo_referenced'
scratch_dir = scratch_dir / 'tmp'
opera.georeference_burst(burst_instance,dem_path,output_dir,scratch_dir)

In [ ]:
!gdalinfo ./scratch/geo_referenced/t106_227662_iw1_20211111_VH.tif